## Create the metadata to alias

In this example, we create two Operators that we will subsequently alias.

In [ ]:
# Perform example-specific setup with the DataStoreContext. This is not needed when writing production code.
from utilities import DataStoreContext
data_store_context = DataStoreContext()
data_store_context.initialize()

from ni.datastore.metadata import MetadataStoreClient, Operator

metadata_store_client = MetadataStoreClient()
operatorOne = Operator(name="Jane Doe")
operatorTwo = Operator(name="John Smith")

metadata_store_client.create_operator(operatorOne)
metadata_store_client.create_operator(operatorTwo)

## Create aliases for the desired metadata

Note: The metadata being aliased must have already been created prior to creating the alias.

In [ ]:
metadata_store_client.create_alias("primary_operator", operatorOne)
metadata_store_client.create_alias("secondary_operator", operatorTwo)

print("Aliases created successfully.")

## Reference the alias

You can reference the alias in place of the metadata ID when creating further metadata or publishing.

In [ ]:
from ni.datastore.data import DataStoreClient, TestResult

test_result = TestResult(
    name="Test Result with Operator Alias",
    operator_id="secondary_operator"
)

data_store_client = DataStoreClient()
test_result_id = data_store_client.create_test_result(test_result)

print("The test results was created successfully using an alias to reference an operator.")


## Verify expected alias usage

In [ ]:
# Retrieve the session and verify which operator was associated with it
retrieved_session = data_store_client.get_test_result(test_result_id)
retrieved_operator_id = retrieved_session.operator_id
operator_details = metadata_store_client.get_operator(retrieved_operator_id)

print(f"The name of the operator for the session: {operator_details.name}")

## Get alias

Look up the target information of the alias (i.e. what the alias is an alias for)

In [ ]:
alias = metadata_store_client.get_alias("primary_operator")

print(alias)

## Query all aliases

In [ ]:
aliases = metadata_store_client.query_aliases("")
for alias in aliases:
    print(alias)

## Query aliases with filter

In [ ]:
aliases = metadata_store_client.query_aliases("$filter=name eq 'primary_operator'")
for alias in aliases:
    print(alias)

## Delete alias

Deletes an existing alias on the system. This does not affect the underlying target/metadata to which the alias refers.

In [ ]:
deleted = metadata_store_client.delete_alias("primary_operator")

print(f"Success of deleting 'primary_operator': {deleted}\n")

## Query the aliases again to confirm the alias has been removed
aliases = metadata_store_client.query_aliases("")
for alias in aliases:
    print(alias)

# Try to delete the alias again. This should not succeed since the alias has already been removed.
deleted = metadata_store_client.delete_alias("primary_operator")
print(f"Success of deleting already-removed 'primary_operator': {deleted}")

## Close the client

In [ ]:
metadata_store_client.close()

# Perform example-specific cleanup. This is not needed when writing production code.
data_store_context.close()